# Procesamiento de los datos

Limpieza y transformaciones, la salida estará lista para modelar.

In [1]:
# settings
import pandas as pd
from   itertools import chain

In [2]:
# data
path_input  = "work/data/celulares.csv"
path_salida = 'work/data/processed/celulares_procesados.csv'

In [3]:
import janitor

In [4]:
# corrigé un error en el formato de los valores de cada instancia
def replace_string(dataframe, string = ','):
  # elimina el caracter molesto
  df = dataframe.copy()
  # column by column
  for columna in df.columns.values:
    df[columna] = df[columna].str.replace(string, '')
  return df 
# lowercase all dataframe
def df_lowercase(dataframe):
  # lowercase all columns
  df = dataframe.copy()
  for columna in df.columns.values:
    df[columna] = df[columna].str.lower()
  return df
# coerse columns
def df_numeric(dataframe, columns):
  df = dataframe.copy()
  df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')
  return df
# agrupo las funciones anteriores
def df_clean(dataframe, string, columns_to_numeric):
  df = dataframe.copy()
  # 
  df_2 = replace_string(dataframe, string)
  df_3 = df_lowercase(df_2)
  df_4 = df_numeric(df_3, columns = columns_to_numeric)
  return df_4

In [5]:
# data loading
df_raw = pd.read_csv(path_input, dtype = 'str').clean_names()

In [6]:
# renombro columnas
nombres = {"nombre_del_producto": 'producto_nombre', 'memoria_interna': 'memoria'}
df_inicio = df_raw.rename(columns = nombres)

In [7]:
# limpieza inicial
columns_numeric = ['peso', 'camara_trasera', 'camara_frontal', 'ram', 'memoria', 'precio']
# 
df_limpio = df_clean(df_inicio, ',', columns_numeric).drop_duplicates().reset_index(drop = True)

## Transformación de las columnas  

Limpieza especifica, o transformaciones e.g `so = {'android', 'ios'}`

In [8]:
# los pasos lod meto en funciones
def clean_tecnologia(dataframe):
    df = dataframe.copy()
    # tabla de soporte
    tabla_tecnologias = pd.DataFrame(
    {'tecnologia' : ['2g/3g/4g/4glte/5g', '4glte',  '4g/gsm', '2g/3g/4g/4glte/gsm', '4g', '5g', '3g/4g/gsm', '4g/4glte/gsm/lte', '2g/3g/lte', '3g/lte'],
    'tecnologia_mejor' : ['5g',            '4glte', '4g',     '4glte',              '4g', '5g', '4g',       '4glte',             '4glte',      '4glte']}
    )
    # sustitución
    df_salida = df.merge(tabla_tecnologias, how = "left").drop(columns = {'tecnologia'}).rename(columns = {'tecnologia_mejor': 'tecnologia'})
    # salida
    return df_salida
    # procesador
def clean_procesador(dataframe):
    df = dataframe.copy()
    #
    df['procesador'] = df.procesador.str.split().str.get(0).str.replace('\d+', '')
    # salida
    return  df
    # clean operative systems
def clean_os(dataframe):
    df = dataframe.copy()
    #
    df['sistema_operativo']= df.sistema_operativo.str.extract(r'(android|ios)', expand = False)
    # salida
    return df
    # chain steps
def df_procesamiento(dataframe):
    df = dataframe.copy()
    # steps 
    df_tecnologia = clean_tecnologia(df)
    df_procesador = clean_procesador(df_tecnologia)
    df_os         = clean_os(df_procesador)
    # resultado
    return df_os

In [9]:
# aplico los pasos
df_procesado = df_procesamiento(df_limpio)

In [10]:
# salvado
df_procesado.to_csv(path_salida, index = False)